In [108]:
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import tree
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt


In [109]:
df = pd.read_csv('laptop_prices.csv')
le = LabelEncoder()

df = df.replace('#NUM!', np.nan)
df = df.fillna(0)

df['Brand'] = le.fit_transform(df['Brand'])
df['Processor'] = le.fit_transform(df['Processor'])
df['Storage'] = le.fit_transform(df['Storage'])
df['GPU'] = le.fit_transform(df['GPU'])
df['Resolution'] = le.fit_transform(df['Resolution'])
df['Operating System'] = le.fit_transform(df['Operating System'])

Y = df['Price ($)'].to_numpy().tolist()
X = df.drop(['Price ($)'], axis=1).to_numpy().tolist()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=20)


In [110]:
model_TREE = GridSearchCV(
    tree.DecisionTreeRegressor(),
    {
        'criterion' : ['squared_error', 'absolute_error'],
        'min_samples_split' : [5,15,17],
        'max_depth' : [4,12,15],
        'min_samples_leaf' : [2,8,15]
    },
    n_jobs = 4
)

In [111]:
model_ENET = GridSearchCV(
    ElasticNet(fit_intercept=False),
    {
        'alpha' : list(map(lambda x : x / 10, range(1,10))),
        'l1_ratio' : list(map(lambda x : x / 10, range(1,10)))
    },
    n_jobs = 4
)

In [112]:
model_SVR = SVR(kernel='linear', degree=4)

In [113]:
model_TREE.fit(X_train, Y_train)
print(model_TREE.best_params_)

{'criterion': 'absolute_error', 'max_depth': 15, 'min_samples_leaf': 15, 'min_samples_split': 17}


In [114]:
model_ENET.fit(X_train, Y_train)
print(model_ENET.best_params_)

{'alpha': 0.1, 'l1_ratio': 0.9}


In [115]:
model_SVR.fit(X_train, Y_train)

SVR(degree=4, kernel='linear')

In [120]:
print('ERRO TRAIN tree:\t',mean_absolute_error(Y_train, model_TREE.predict(X_train)))
print('ERRO TEST  tree:\t',mean_absolute_error(Y_test, model_TREE.predict(X_test)))
print()
print('ERRO TRAIN enet:\t',mean_absolute_error(Y_train, model_ENET.predict(X_train)))
print('ERRO TEST  enet:\t',mean_absolute_error(Y_test, model_ENET.predict(X_test)))
print()
print('ERRO TRAIN  svr:\t',mean_absolute_error(Y_train, model_SVR.predict(X_train)))
print('ERRO TEST   svr:\t',mean_absolute_error(Y_test, model_SVR.predict(X_test)))

ERRO TRAIN tree:	 316.4565402591884
ERRO TEST  tree:	 361.1000488530162

ERRO TRAIN enet:	 591.8611309434388
ERRO TEST  enet:	 606.1341309653118

ERRO TRAIN  svr:	 573.7285576555029
ERRO TEST   svr:	 590.2474276142325
